## Importar Librerias

In [1]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
# Puede que nos sirvan también
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, Perceptron, Ridge
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, classification_report, roc_curve, auc
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)  # Para mayor determinismo

## 1.Carga de datos

Al final del practico anterior obtuvimos un dataset enriquesido con features georeferenciadas. Es el momento de utilizarlo.

Exporta el dataframe obtenido en el ultimo practico a un csv y cargalo como dataframe.

In [11]:
dataframe = pd.read_csv('./data/post_curation.csv', sep=';')
dataframe

,COD_ID,DIST,PAC_1,PAC_2,PAC_3,FAS_CON_P,FAS_CON_S,FAS_CON_T,SIT_ATIV,TIP_UNID,...,n_close_con,is_urb,closest_trd_i,closest_trd_dist,NODE,ENDNODE,g_degree,g_avg_ndeg,g_pagerank,dist_to_trs
0,4_1040341,46,452971,ET4_1040341,0,ABC,ABCN,0,AT,38,...,1,False,2054,182.473575,45297,44304,2,2.0,0.000012,8170.23
1,4_1040306,46,452681,ET4_1040306,0,ABC,ABCN,0,AT,38,...,1,False,4111,668.681912,45268,44304,1,2.0,0.000007,10274.81
2,7_2070013,46,600021,ET7_2070013,0,ABC,ABCN,0,AT,38,...,195,True,6269,35.663825,60002,48689,1,3.0,0.000006,1090.47
3,4_1040035,46,462831,ET4_1040035,ET4_1040035,AB,AN,BN,AT,38,...,14,False,8962,191.940500,46283,44304,2,2.0,0.000012,15507.34
4,13_1130368,46,148261,ET13_1130368,0,ABC,ABCN,0,AT,38,...,4,False,5214,58.166348,14826,17749,2,2.5,0.000011,29516.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499,14_1141247,46,6800221,ET14_1141247,ET14_1141247,AB,AN,BN,AT,38,...,1,False,8633,384.548354,680022,17749,1,2.0,0.000007,35574.06
10500,8_1080557,46,6800431,ET8_1080557,0,ABC,ABCN,0,AT,38,...,4,False,5013,112.433295,680043,71128,1,2.0,0.000007,10719.20
10501,2_1020110,46,6800681,ET2_1020110,0,ABC,ABCN,0,AT,38,...,1,False,9354,182.742433,680068,118082,1,2.0,0.000007,18159.75
10502,9_2090010,46,6775691,ET9_2090010,ET9_2090010,AB,AN,BN,AT,38,...,1,False,4543,341.431358,677569,68429,1,3.0,0.000006,22236.72


Ahora debemos eliminar variables que no aporte informacion por ejemplo:


*   Indices y IDs
*   Referencias a otras tablas
*   Variables compuestas (listas, jsons, geometrias)

Identifica las variables que cumplen este criterio y eliminelas del dataset. Retira tambien la variable de salida que no van a utilizar FIC o DIC.



In [12]:
# Elimino índices, IDs y variables compuestas.
dataframe = dataframe.drop(columns=['COD_ID', 'PAC_1', 'PAC_2', 'geometry'])

# Elimino las siguientes variables dado que:
# DIST == 46 para todas las filas,
# CONF == 'RA' para todas las filas excepto una,
# POSTO == 'PT' para todas las filas,
# BANC == 0 para todas la filas,
# MRT == 0 para todas la filas,
# DESCR = '' para todas las filas,
# eu_dis_trs = -1 para todas las filas.
dataframe = dataframe.drop(columns=['DIST', 'CONF', 'POSTO', 'BANC', 'MRT', 'DESCR', 'eu_dis_trs'])

# Con respecto a otras variables:
# DAT_CON es una fecha. Nos sirve?
# Qué es NODE? No me acuerdo...
dataframe

,PAC_3,FAS_CON_P,FAS_CON_S,FAS_CON_T,SIT_ATIV,TIP_UNID,POS,ATRB_PER,TEN_LIN_SE,CAP_ELO,...,n_close_con,is_urb,closest_trd_i,closest_trd_dist,NODE,ENDNODE,g_degree,g_avg_ndeg,g_pagerank,dist_to_trs
0,0,ABC,ABCN,0,AT,38,PD,1,0.22,05H,...,1,False,2054,182.473575,45297,44304,2,2.0,0.000012,8170.23
1,0,ABC,ABCN,0,AT,38,PD,1,0.22,05H,...,1,False,4111,668.681912,45268,44304,1,2.0,0.000007,10274.81
2,0,ABC,ABCN,0,AT,38,O,2,0.38,5H,...,195,True,6269,35.663825,60002,48689,1,3.0,0.000006,1090.47
3,ET4_1040035,AB,AN,BN,AT,38,PD,1,0.23,1H,...,14,False,8962,191.940500,46283,44304,2,2.0,0.000012,15507.34
4,0,ABC,ABCN,0,AT,38,PD,1,0.38,2H,...,4,False,5214,58.166348,14826,17749,2,2.5,0.000011,29516.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499,ET14_1141247,AB,AN,BN,AT,38,PD,1,0.44,05H,...,1,False,8633,384.548354,680022,17749,1,2.0,0.000007,35574.06
10500,0,ABC,ABCN,0,AT,38,PD,1,0.22,2H,...,4,False,5013,112.433295,680043,71128,1,2.0,0.000007,10719.20
10501,0,ABC,ABCN,0,AT,38,PD,1,0.22,1H,...,1,False,9354,182.742433,680068,118082,1,2.0,0.000007,18159.75
10502,ET9_2090010,AB,AN,BN,AT,38,O,1,0.23,05H,...,1,False,4543,341.431358,677569,68429,1,3.0,0.000006,22236.72


## 2. Preprosecamiento y particion de datos

Siempre se recomienda partir los datos antes de realizar el preprocesamiento, y guardar el pipeline de transformacion para aplicar en operaciones futuras.

Sin embargo algunas operaciones principalmente la codificacion de variables puede realizarse antes de hacer la particion ya que no representa una interaccion entre las filas.

Genere una particion de datos en sets de entrenamiento y validacion.

Un elemento que tenemos que siempre debemos tener en cuenta es que el procesamiento que debemos hacer sobre los datos esta relacionado con el algoritmo que vayas a implementar.

## 3. Preparacion de datos para regresion lineal

Los algoritmos de regresion solo aceptan entradas numericas, por lo tanto el primer paso que debemos realizar es codificar las variables categoricas numericamente.

Adicionalmente las diferencias en las dimensiones absolutas de las variables van a tener un impacto muy pronunciado en los coeficientes calculados por lo que tambien se recomienda realizar la normalizacion de las variables. 

Esto puede hacerse de varias maneras:

*   Media/Mediana y desviacion standard
*   Minimo y Maximo (Cuidado con los outliers)
*   De 1 a -1

Implemente una funcion o pipeline que aplique codificacion de variables categoricas y luego normalizacion. Recuerde que los parametros de normalizacion se calculan usando unicamente los valores de entrenamiento.

## 4. Regresion lineal simple y regularizada.

Con los datos preprocesados obtenidos en el punto anterior implemente una regresion lineal. 

### Evaluacion del modelo

Para evaluar el modelo calcule R cuadrado, error medio absoluto y error medio cuadrado. Tanto de entrenamiento como de validacion.

### Analisis de residuos

Otro punto importante en la evaluacion de los modelos de regresion, es no solo saber cuanto es el error sino como se distribuye. Para eso realizamos un analisis de residuos. Los residuos son simplemente las desviaciones de la prediccion al valor real.

Primero grafique la distribucion de los residuos.

Esta distribucion debe estar centrada en cero y deberia tender a ser simetrica.

**A continuacion genere un scatter plot entre la prediccion y el error.** 

Esto deberia verse principalmente como ruido, ya que no deberia haber correlacion entre estos valores.

Finalmente el ultimo grafico que realizaremos es el QQ Plot, que nos indicara la distribucion del error a lo largo de los cuartiles.

**Genere un qq plot con las predicciones realizadas.**
(from statsmodels.graphics.gofplots import qqplot)


En este caso los valores deben concentrarse sobre la linea diagonal.

#### Opcional:

La regularizacion es un metodo mediante el cual se penalisa el valor obsoluto de los coeficientes, de este modo se encuentran modelos mas estables y menos propensos a overfiear los datos.

Implemente una Regresion tipo Lasso (Regularizacion tipo L1) o tipo Ridge (Regularizacion tipo L2), calcula sus metricas y compare con la regresion lineal comun.

## 5. Importancia de las variables, Regresion Lineal

En base a la arquitectura del modelo y utilizando los coeficientes del modelo, determine cuales son las 10 variables de mayor peso en el modelo y explique porque.

## 6. Preparacion de datos para algoritmos basados en arboles

Otra rama muy utilizada de modelos son aquellos basados en arboles de decision, como Bagging, Random Forrest o Boosting. En nuestro caso utilizaremos una implementacion de Boosting por gradiente denominada XGBoost que es reconocida por producir muy buenos resultados.

Dada la naturaleza de los arboles de decision en este caso es necesario normalizar los datos. Para demostrarlo, entrene dos modelos con parametros por default, utilizando el dataset normalizado y sin normalizar.

### Evaluacion del modelo, XGBoost

Seleccione uno de los modelos, calcule las metricas y realize el analisis un analisis de residuos, similar al realizado anteriormente.

## 7. Importancia de las variables

Utilize el metodo "Feature Importance" para determinar las 10 variables mas importante del modelo, verifique los diferentes criterios de calculo, seleccione uno y grafique sus coeficientes.

## 8. (Opcional) Conversion a problema de clasificacion

Podemos cambiar la naturaleza del problema de regresion a clasificacion simplemente cuantizando la salida y asignando una categoria cada intervalo.

Separe la salida en 5 intervalos segun su criterio, asignele categorias de 0 a 5, siendo 0 la peor calidad y 5 la mejor.

Entrene un classificador XGBoost, y calcule las metricas de accuracy, precision, recall, f1 score.

## 9. Conclusiones

A partir de todos los puntos anteriores exponga sus impresiones y conclusiones personales.